# Fetching Music Data with Spotipy and Lyricsgenius

## Import libraries

In [7]:
import spotipy
import pandas as pd
import lyricsgenius as lg
import re
from spotipy.oauth2 import SpotifyClientCredentials

## Getting Song Infos From the Spotify API

In [8]:
# Credentials from your developer account
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="c297bfcba2534de3ba91cf03756bb185",
                                                           client_secret="36f7108debd34e34bffa6941b9e508a1"))

In [9]:
# Search for Radiohead's artist information
artist_name = "Radiohead"
results = sp.search(q=artist_name, type='artist')
artist = results['artists']['items'][0]  # Assuming Radiohead is the first result
artist_id = artist['id']

# Create an empty list to store the data for each track
data = []

# Retrieve the list of albums by Radiohead
albums = sp.artist_albums(artist_id, album_type='album')

# Loop through the albums and retrieve audio features for each track
for album in albums['items']:
    album_name = album['name']
    album_id = album['id']
    tracks = sp.album_tracks(album_id)
    for track in tracks['items']:
        track_name = track['name']
        track_id = track['id']
        track_number = track['track_number']
        duration_ms = track['duration_ms']
        release_year = album['release_date'].split('-')[0]  # Extract the release year

        # Get audio features for the track
        audio_features = sp.audio_features([track_id])

        if audio_features:
            audio_feature_data = audio_features[0]

            # Append the data for each track to the list
            data.append([artist_name, album_name, track_name, track_number, duration_ms, release_year,
                         audio_feature_data['valence'], audio_feature_data['energy'],
                         audio_feature_data['danceability'], audio_feature_data['acousticness'],
                         audio_feature_data['instrumentalness'], audio_feature_data['speechiness'],
                         audio_feature_data['liveness'], audio_feature_data['tempo'],
                         audio_feature_data['key'], audio_feature_data['mode'], audio_feature_data['time_signature']])

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Artist', 'Album', 'Track', 'TrackNumber_On_Album', 'Duration_ms', 'ReleaseYear', 'Valence', 'Energy', 'Danceability', 'Acousticness', 'Instrumentalness', 'Speechiness', 'Liveness', 'Tempo', 'Key', 'Mode', 'TimeSignature'])




Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=797AyTQcqoQgqWQ1Qz184Y:
 Max Retries, reason: too many 429 error responses

In [ ]:
#Filter out the studio albums

radiohead_albums = [
    "Pablo Honey",
    "The Bends",
    "OK Computer",
    "Kid A",
    "Amnesiac",
    "Hail To the Thief",
    "In Rainbows",
    "The King Of Limbs",
    "A Moon Shaped Pool"
]

df = df[df['Album'].isin(radiohead_albums)]

,Artist,Album,Track,TrackNumber_On_Album,Duration_ms,ReleaseYear,Valence,Energy,Danceability,Acousticness,Instrumentalness,Speechiness,Liveness,Tempo,Key,Mode,TimeSignature,lyrics
57,Radiohead,A Moon Shaped Pool,Burn the Witch,1,220609,2016,0.620,0.847,0.540,0.303000,0.2720,0.0297,0.109,148.936,11,1,4,stay in the shadows cheer at the gallows this ...
58,Radiohead,A Moon Shaped Pool,Daydreaming,2,384438,2016,0.113,0.263,0.214,0.968000,0.8530,0.0336,0.126,137.561,9,0,3,dreamers they never learn they never learn bey...
59,Radiohead,A Moon Shaped Pool,Decks Dark,3,281011,2016,0.271,0.501,0.557,0.666000,0.8370,0.0269,0.117,139.149,2,0,4,then into your life there comes a darkness the...
60,Radiohead,A Moon Shaped Pool,Desert Island Disk,4,224698,2016,0.158,0.282,0.560,0.868000,0.5290,0.0287,0.110,112.009,0,1,1,now as i go upon my way so let me go upon my w...
61,Radiohead,A Moon Shaped Pool,Ful Stop,5,367348,2016,0.178,0.662,0.425,0.352000,0.9410,0.0502,0.619,151.182,0,1,3,you really messed up everything you really mes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Radiohead,Pablo Honey,Vegetable,8,192666,1993,0.419,0.717,0.382,0.002810,0.5690,0.0339,0.187,105.820,6,1,4,i never wanted anything but this i worked hard...
189,Radiohead,Pablo Honey,Prove Yourself,9,145373,1993,0.319,0.620,0.250,0.000849,0.0848,0.0611,0.175,114.097,0,1,4,i can't afford to breathe in this time nowhere...
190,Radiohead,Pablo Honey,I Can't,10,253093,1993,0.273,0.825,0.270,0.009850,0.3080,0.0607,0.118,106.147,7,1,4,please forget the words that i just blurted ou...
191,Radiohead,Pablo Honey,Lurgee,11,187866,1993,0.409,0.548,0.420,0.000135,0.3990,0.0268,0.111,101.888,0,1,4,i feel better i feel better now you've gone i ...


## Adding the lyrics from Genius Lyrics

In [ ]:
#Use your API key from your developer setup
api_key = '2JmbZjrQqNS_AC1iSo3hUoNKEweXvBsYiS6A9w6CAXr-ds8qWbo0G6663345IRkH'

#Set up the formatting of the lyrics
genius = lg.Genius(api_key, response_format= 'plain', remove_section_headers= True, excluded_terms=["(Remix)", "(Live)"] )

In [ ]:
#Function to fetch lyrics

def fetch_lyrics(title, artist):
    # Use Genius API to search for lyrics
    song = genius.search_song(title, artist)
    if song:
        return song.lyrics
    else:
        return None

In [ ]:
#Fetching lyrics for every song in the data frame

df['Lyrics'] = df.apply(lambda row: fetch_lyrics(row['Track'], row['Artist']), axis=1)


NameError: name 'df' is not defined

In [ ]:
#Checking for missing lyrics
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 57 to 192
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Artist                101 non-null    object 
 1   Album                 101 non-null    object 
 2   Track                 101 non-null    object 
 3   TrackNumber_On_Album  101 non-null    int64  
 4   Duration_ms           101 non-null    int64  
 5   ReleaseYear           101 non-null    object 
 6   Valence               101 non-null    float64
 7   Energy                101 non-null    float64
 8   Danceability          101 non-null    float64
 9   Acousticness          101 non-null    float64
 10  Instrumentalness      101 non-null    float64
 11  Speechiness           101 non-null    float64
 12  Liveness              101 non-null    float64
 13  Tempo                 101 non-null    float64
 14  Key                   101 non-null    int64  
 15  Mode                  101 n

In [ ]:
#Confirming that these are indeed purely instrumental tracks
df.loc[df.loc[:,'Lyrics'].isna()]

,Artist,Album,Track,TrackNumber_On_Album,Duration_ms,ReleaseYear,Valence,Energy,Danceability,Acousticness,Instrumentalness,Speechiness,Liveness,Tempo,Key,Mode,TimeSignature,lyrics
143,Radiohead,Amnesiac,Hunting Bears,9,121200,2001,0.0736,0.264,0.295,0.853,0.926,0.0432,0.0962,143.191,7,1,3,None
150,Radiohead,Kid A,Treefingers,5,222600,2000,0.0577,0.146,0.165,0.827,0.887,0.0362,0.1090,134.508,6,1,3,None
156,Radiohead,Kid A,Untitled,11,52694,2000,0.0769,0.225,0.369,0.992,0.813,0.0654,0.1060,64.655,7,1,3,None


In [ ]:
#Function to clean lyrics

def clean_lyrics(lyrics):
    if lyrics is None:
        return None
    
    # Use the str.find and str.slice methods to extract the lyrics
    lyrics_start = lyrics.find('Lyrics\n')
    
    if lyrics_start != -1:
        cleaned_lyrics = lyrics[lyrics_start + len('Lyrics\n'):]
        # Make lyrics lowercase, replace '\n' with ' ', and remove two random digits + 'embed' at the end
        cleaned_lyrics = cleaned_lyrics.lower().replace('\n', ' ')
        
        # Use a regular expression to remove 'embed' with one, two, or three digits in front of it
        cleaned_lyrics = re.sub(r'(\d{1,3}embed\b)', '', cleaned_lyrics)

        
        # Remove zero-width space and 'you might also like'
        cleaned_lyrics = cleaned_lyrics.replace('\u200b', '').replace('you might also like', '')

        # Remove all punctuation except for the single quote
        cleaned_lyrics = re.sub(r'[^\w\s\']', '', cleaned_lyrics)

    else:
        cleaned_lyrics = lyrics

    return cleaned_lyrics

In [ ]:
#Applying cleaning function

df['Lyrics'] = df['Lyrics'].apply(clean_lyrics)


In [ ]:
# Save dataframe to a CSV file
df.to_csv('radiohead_spotify_lyricsgenius.csv', index=False)